# **1 - Model Inference**

## 1.1. Import Libraries

In [8]:
# Import libraries
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import tensorflow as tf
import re
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from keras.models import load_model
import string

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1.2. Load the Model

In [3]:
# Extract the model
!unzip -q /content/StressPrediction.zip

In [4]:
# Load the model
loadModel = tf.keras.models.load_model('/content/StressPrediction')

# **2 - Inferencing**

In [6]:
# Create a new data
dataInf = {
    'text' : 'In the uncertainty and pressure she faces, she feels paralyzing stress, so fatigue and anxiety merge into one.',
    'label' : '1'
}

dataInf = pd.DataFrame([dataInf])
dataInf

,text,label
0,"In the uncertainty and pressure she faces, she...",1


In [11]:
# Define stopwords
engStopwords = stopwords.words("english")

# New Stopwords
newStopwords = ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don't",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"
 ]

# Compile Stopwords
stw_en = engStopwords + newStopwords
stw_en = list(set(stw_en))

In [12]:
# Define stemmer
lanc = LancasterStemmer()

# Function for text pre-processing
def textProcess(text):
  # Change to lowercase
  text = text.lower()

  # Remove hashtag
  text = re.sub('#[A-Za-z0-9_]+', ' ', text)

  # Remove \n
  text = re.sub(r'\\n', ' ', text)

  # Remove whitespace
  text = text.strip()

  # Remove link
  text = re.sub(r'http\S+', ' ', text)
  text = re.sub(r'www.\S+', ' ', text)

  # Remove duplicate character
  text = re.sub('(.)\\1{2,}', '\\1', text)

  # Remove not char, emoji etc
  text = re.sub("[^A-Za-z\s']", " ", text)

  # Spacing cleaning
  text = ' '.join(text.split())

  # Tokenize
  tokens = word_tokenize(text)

  # Remove Stopwords
  text = ' '.join([word for word in tokens if word not in engStopwords])
  
  # Melakukan Stemming
  text = lanc.stem(text)
  
  return text

# Define Lemmatizer
def lemmatize_text(text) : 
  sentence = []
  for word in text.split():
    lemmatizer = WordNetLemmatizer()
    sentence.append(lemmatizer.lemmatize(word, 'v'))
  return ' '.join(sentence)

In [13]:
# Applying pre-processing into data inference
dataInf['text_processed'] = dataInf['text'].apply(textProcess)
dataInf

,text,label,text_processed
0,"In the uncertainty and pressure she faces, she...",1,uncertainty pressure faces feels paralyzing st...


In [14]:
# Review in lemmatizer
nltk.download('wordnet')
dataInf['text_processed'] = dataInf['text_processed'].apply(lemmatize_text)
dataInf

[nltk_data] Downloading package wordnet to /root/nltk_data...


,text,label,text_processed
0,"In the uncertainty and pressure she faces, she...",1,uncertainty pressure face feel paralyze stress...


In [15]:
# Define Inference
inf = dataInf['text_processed']

In [17]:
# Apply model prediction
%%time
predInf = loadModel.predict(inf)
predInf = np.where(predInf >= 0.5, 1, 0)
predInf

1/1 [==============================] - 2s 2s/step
CPU times: user 2 s, sys: 30 ms, total: 2.03 s
Wall time: 2.01 s


array([[1]])

Hasil dari model mengindikasikan bahwa kalimat yang dibuat pada model inference menunjukan bahwa seseorang sedang dalam tahap stress. 